In [ ]:
# Libraries
import numpy as np
import pandas as pd
import time
import scipy.sparse as spr
from scipy.optimize import minimize
from sklearn import linear_model

In [ ]:
# File
thepath = 'https://raw.githubusercontent.com/math-econ-code/mec_optim_2021-01/master/data_mec_optim/gravity_wtodata/'
tradedata = pd.read_csv(thepath + '1_TraditionalGravity_from_WTO_book.csv')

# Formatting
tradedata = tradedata[['exporter', 'importer','year', 'trade', 'DIST','ln_DIST', 'CNTG', 'LANG', 'CLNY']]
tradedata.sort_values(by = ['year','importer','exporter'], inplace = True)
tradedata.reset_index(inplace = True, drop = True)

# Initiating matrices
nbt = len(tradedata['year'].unique())
nbi = len(tradedata['importer'].unique())
nbk = 4
years = tradedata['year'].unique()
distances = np.array(['ln_DIST', 'CNTG', 'LANG', 'CLNY'])

phi_i_j_t_k = np.zeros((nbi, nbi, nbt, nbk)) # design matrix
tradevol_i_j_t = np.zeros((nbi, nbi, nbt)) # dependent variable (trade volumes between each pair of countries across year)
pihat_i_j_t = np.zeros((nbi, nbi, nbt)) # normalization of the trade volumes

# Filling matrices using data points from tradedata
for t, year in enumerate(years):
    tradevol_i_j_t[:, :, t] = np.array(tradedata.loc[tradedata['year'] == year, 'trade']).reshape((nbi, nbi))  # store trade flows
    np.fill_diagonal(tradevol_i_j_t[:, :, t], 0)  # set to zero the trade within a country
    for k, distance in enumerate(distances):
        phi_i_j_t_k[:, :, t, k] = np.array(tradedata.loc[tradedata['year'] == year, distance]).reshape((nbi, nbi))  # store distances
pihat_i_j_t = tradevol_i_j_t / (tradevol_i_j_t.sum() / len(years)) # normalize by the average annual trade volume

# Keeping only the first year of data
phi_i_j_k = phi_i_j_t_k[:, :, 0, :]
pihat_i_j = pihat_i_j_t[:, :, 0]

In [ ]:
# Self-defined procedure

(I, J, K) = phi_i_j_k.shape

def mloglikelihood(params):

    # Splitting parameters
    lambda_k = params[ : K]
    u_i = params[K : K+I]
    v_j = params[K+I : ]

    # Defining terms for expression from q2 a
    term1 = (pihat_i_j * ((phi_i_j_k * lambda_k[None, None, :]).sum(axis = 2) - u_i[:, None] - v_j[None, :])).sum(axis = (0,1))
    term2 = np.exp((phi_i_j_k * lambda_k[None, None, :]).sum(axis = 2) - u_i[:, None] - v_j[None, :]).sum(axis = (0,1))

    return term2 - term1

def grad_mloglikelihood(params):

    # Splitting parameters
    lambda_k = params[ : K]
    u_i = params[K : K+I]
    v_j = params[K+I : ]

    # Defining equality constraints to be satisfied, as stated in q2 b
    exp_i_j = np.exp((phi_i_j_k * lambda_k[None, None, :]).sum(axis = 2) - u_i[:, None] - v_j[None, :])

    grad_u = (pihat_i_j - exp_i_j).sum(axis = 1) # eq. 1 from q2 b
    grad_v = (pihat_i_j - exp_i_j).sum(axis = 0) # eq. 2 from q2 b
    grad_lambda = ((exp_i_j - pihat_i_j)[:, :, None] * phi_i_j_k).sum(axis = (0, 1)) # eq. 3 from q2 b

    return np.concatenate([grad_lambda, grad_u, grad_v])

# Initiating vectors for regressors + i-fixed effects + j-fixed effects
lambda0_k = np.zeros(K)
u0_i = np.zeros(I)
v0_j = np.zeros(J)
all_params = np.concatenate([lambda0_k, u0_i, v0_j])

result = minimize(mloglikelihood, all_params, jac = grad_mloglikelihood, method = 'BFGS') # minimizing as the input is negative of loglikelihood

# Scikit-learn's pre-defined procedure
clf = linear_model.PoissonRegressor( alpha = 0,  fit_intercept = False ,max_iter = 2000, tol = 1e-10)
X = np.block([[phi_i_j_k.reshape((-1,K)), - np.kron(np.eye(I),np.ones((J,1))), - np.kron(np.ones((J,1)), np.eye(I))]])
clf.fit(X, pihat_i_j.flatten())

# Comparison between the two
print(f'Parameters a/c to own code = {result.x[:K]}')
print(f'Parameters a/c to sk-learn = {clf.coef_[:K]}')

Parameters a/c to own code = [-0.23013921  1.38532591  0.48408635 -0.09585853]
Parameters a/c to sk-learn = [-0.23001125  1.38547253  0.48440474 -0.0960905 ]
